In [ ]:
import nltk

In [ ]:
nltk.download('punkt_tab')


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [ ]:
from nltk.stem.porter import PorterStemmer
stemmer=PorterStemmer()

In [ ]:
def tokenize(sentence):
    """
    split sentence into array of words/tokens
    a token can be a word or punctuation character, or number
    """
    return nltk.word_tokenize(sentence)

In [ ]:
def stem(word):
    """
    stemming = find the root form of the word
    examples:
    words = ["organize", "organizes", "organizing"]
    words = [stem(w) for w in words]
    -> ["organ", "organ", "organ"]
    """
    return stemmer.stem(word.lower())

In [ ]:
a="How long does shipping take?"
print(a)
a=tokenize(a)
print(a)

How long does shipping take?
['How', 'long', 'does', 'shipping', 'take', '?']


In [ ]:
def bag_of_words(tokenized_sentence, words):
    """
    return bag of words array:
    1 for each known word that exists in the sentence, 0 otherwise
    example:
    sentence = ["hello", "how", "are", "you"]
    words = ["hi", "hello", "I", "you", "bye", "thank", "cool"]
    bog   = [  0 ,    1 ,    0 ,   1 ,    0 ,    0 ,      0]
    """
    # stem each word
    sentence_words = [stem(word) for word in tokenized_sentence]
    # initialize bag with 0 for each word
    bag = np.zeros(len(words), dtype=np.float32)
    for idx, w in enumerate(words):
        if w in sentence_words:
            bag[idx] = 1

    return bag

**Model**

In [ ]:
import torch
import torch.nn as nn

In [ ]:
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.l1 = nn.Linear(input_size, hidden_size)
        self.l2 = nn.Linear(hidden_size, hidden_size)
        self.l3 = nn.Linear(hidden_size, num_classes)
        self.relu = nn.ReLU()

    def forward(self, x):
        out = self.l1(x)
        out = self.relu(out)
        out = self.l2(out)
        out = self.relu(out)
        out = self.l3(out)
        # no activation and no softmax at the end
        return out

**Training**

In [ ]:
# import json
# with open('intents.json','r') as f:
#   intents=json.load(f)

#   print(intents)

In [ ]:
import json
with open('intents.json','r') as f:
  intents=json.load(f)

  print(intents)

{'intents': [{'tag': 'admissions', 'patterns': ['How can I apply for admission?', 'What are the admission requirements?', 'When is the application deadline?', 'Can I apply online?'], 'responses': ['You can apply for admission through our online portal. Visit the Admissions section on our website for more details.', 'Admission requirements vary by program. Please check the program-specific details in the Admissions section.', 'The application deadline depends on the program. Please refer to the academic calendar on the website.', 'Yes, you can apply online by visiting the Admissions section on our website.']}, {'tag': 'courses', 'patterns': ['What courses are offered?', 'Do you have engineering programs?', 'Can I see a list of courses?', 'What are the popular programs?'], 'responses': ['We offer a wide range of programs, including engineering, business, arts, and sciences. Visit the Programs section for a detailed list.', 'Yes, we have engineering programs. You can choose from various s

In [ ]:
all_words=[]
tags=[]
xy=[]

for intent in intents['intents']:
  tag=intent['tag']
  tags.append(tag)
  for pattern in intent['patterns']:
    w=tokenize(pattern)
    all_words.extend(w)
    xy.append((w,tag))

# stem and lower each word
ignore_words = ['?', '.', '!']
all_words = [stem(w) for w in all_words if w not in ignore_words]
# remove duplicates and sort
all_words = sorted(set(all_words))
tags = sorted(set(tags))

print(len(xy), "patterns")
print(len(tags), "tags:", tags)
print(len(all_words), "unique stemmed words:", all_words)

56 patterns
14 tags: ['academic_calendar', 'admissions', 'clubs', 'contact', 'courses', 'events', 'facilities', 'fees', 'hostel', 'internships', 'library', 'placement', 'scholarships', 'transportation']
107 unique stemmed words: ['a', 'about', 'academ', 'accommod', 'activ', 'address', 'admiss', 'aid', 'an', 'and', 'ani', 'appli', 'applic', 'are', 'avail', 'book', 'borrow', 'boy', 'bu', 'calendar', 'campu', 'can', 'cell', 'charg', 'club', 'colleg', 'compani', 'complex', 'contact', 'cost', 'cours', 'cultur', 'deadlin', 'do', 'doe', 'email', 'engin', 'event', 'facil', 'fee', 'fest', 'financi', 'find', 'for', 'from', 'girl', 'happen', 'have', 'help', 'holiday', 'hostel', 'how', 'i', 'inquiri', 'internship', 'is', 'librari', 'list', 'mandatori', 'mba', 'me', 'merit-bas', 'much', 'next', 'number', 'of', 'offer', 'offic', 'on', 'onlin', 'opportun', 'organ', 'phone', 'placement', 'popular', 'process', 'program', 'provid', 'reach', 'requir', 'schedul', 'scholarship', 'see', 'semest', 'seminar',

In [ ]:
import numpy as np

In [ ]:
# create training data
X_train = []
y_train = []
for (pattern_sentence, tag) in xy:
    # X: bag of words for each pattern_sentence
    bag = bag_of_words(pattern_sentence, all_words)
    X_train.append(bag)
    # y: PyTorch CrossEntropyLoss needs only class labels, not one-hot
    label = tags.index(tag)
    y_train.append(label)

X_train = np.array(X_train)
y_train = np.array(y_train)


In [ ]:
# print(X_train)
# print(y_train)

In [ ]:
# pip install model

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

In [ ]:
# Hyper-parameters
num_epochs = 1000
batch_size = 8
learning_rate = 0.001
input_size = len(X_train[0])
hidden_size = 8
output_size = len(tags)
print(input_size, output_size)


107 14


In [ ]:
class ChatDataset(Dataset):

    def __init__(self):
        self.n_samples = len(X_train)
        self.x_data = X_train
        self.y_data = y_train

    # support indexing such that dataset[i] can be used to get i-th sample
    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    # we can call len(dataset) to return the size
    def __len__(self):
        return self.n_samples

dataset = ChatDataset()
train_loader = DataLoader(dataset=dataset,
                          batch_size=batch_size,
                          shuffle=True,
                          num_workers=0)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = NeuralNet(input_size, hidden_size, output_size).to(device)

In [ ]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)


In [ ]:
# Train the model
for epoch in range(num_epochs):
    for (words, labels) in train_loader:
        words = words.to(device)
        labels = labels.to(dtype=torch.long).to(device)

        # Forward pass
        outputs = model(words)
        # if y would be one-hot, we must apply
        # labels = torch.max(labels, 1)[1]
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    if (epoch+1) % 100 == 0:
        print (f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')


print(f'final loss: {loss.item():.4f}')

Epoch [100/1000], Loss: 0.6870
Epoch [200/1000], Loss: 0.0417
Epoch [300/1000], Loss: 0.0283
Epoch [400/1000], Loss: 0.0052
Epoch [500/1000], Loss: 0.0047
Epoch [600/1000], Loss: 0.0015
Epoch [700/1000], Loss: 0.0009
Epoch [800/1000], Loss: 0.0009
Epoch [900/1000], Loss: 0.0003
Epoch [1000/1000], Loss: 0.0003
final loss: 0.0003


In [ ]:
data = {
"model_state": model.state_dict(),
"input_size": input_size,
"hidden_size": hidden_size,
"output_size": output_size,
"all_words": all_words,
"tags": tags
}

In [ ]:
# FILE = "data.pth"
# torch.save(data, FILE)

# print(f'training complete. file saved to {FILE}')

training complete. file saved to data.pth


In [ ]:
FILE = "genai05.pth"
torch.save(data, FILE)

print(f'training complete. file saved to {FILE}')

training complete. file saved to genai05.pth


**chat**

In [ ]:
import random
import json

import torch



device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

with open('intents.json', 'r') as json_data:
    intents = json.load(json_data)

FILE = "genai05.pth"
data = torch.load(FILE)


<ipython-input-73-560f4911eb5b>:14: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  data = torch.load(FILE)


change ['intents'] to requires json file name

In [ ]:
input_size = data["input_size"]
hidden_size = data["hidden_size"]
output_size = data["output_size"]
all_words = data['all_words']
tags = data['tags']
model_state = data["model_state"]

model = NeuralNet(input_size, hidden_size, output_size).to(device)
model.load_state_dict(model_state)
model.eval()

bot_name = "Sam"
print("Let's chat! (type 'quit' to exit)")
while True:
    # sentence = "do you use credit cards?"
    sentence = input("You: ")
    if sentence == "quit":
        break

    sentence = tokenize(sentence)
    X = bag_of_words(sentence, all_words)
    X = X.reshape(1, X.shape[0])
    X = torch.from_numpy(X).to(device)

    output = model(X)
    _, predicted = torch.max(output, dim=1)

    tag = tags[predicted.item()]

    probs = torch.softmax(output, dim=1)
    prob = probs[0][predicted.item()]
    if prob.item() > 0.75 :
        for intent in intents['intents']:
            if tag == intent["tag"]:
                print(f"{bot_name}: {random.choice(intent['responses'])}")
    else:
        print(f"{bot_name}: I do not understand...")

Let's chat! (type 'quit' to exit)
You: quit
